# Generatiiviset verkot

Toistuvat neuroverkot (RNN:t) ja niiden portitetut solumuunnelmat, kuten Long Short Term Memory -solut (LSTM:t) ja Gated Recurrent Units (GRU:t), tarjoavat mekanismin kielen mallintamiseen, eli ne voivat oppia sanajärjestyksen ja ennustaa seuraavan sanan sekvenssissä. Tämä mahdollistaa RNN:ien käytön **generatiivisissa tehtävissä**, kuten tavallisessa tekstin generoinnissa, konekäännöksessä ja jopa kuvatekstien luomisessa.

RNN-arkkitehtuurissa, jota käsittelimme edellisessä osiossa, jokainen RNN-yksikkö tuotti seuraavan piilotetun tilan ulostulona. Voimme kuitenkin lisätä jokaiselle toistuvalle yksikölle toisen ulostulon, joka mahdollistaa **sekvenssin** tuottamisen (joka on yhtä pitkä kuin alkuperäinen sekvenssi). Lisäksi voimme käyttää RNN-yksiköitä, jotka eivät ota syötettä jokaisessa vaiheessa, vaan ainoastaan jonkin alkuperäisen tilavektorin, ja tuottavat sitten ulostulosekvenssin.

Tässä muistikirjassa keskitymme yksinkertaisiin generatiivisiin malleihin, jotka auttavat meitä tuottamaan tekstiä. Yksinkertaisuuden vuoksi rakennetaan **merkki-tason verkko**, joka tuottaa tekstiä kirjain kerrallaan. Koulutuksen aikana meidän täytyy ottaa jokin tekstikorpus ja jakaa se kirjainsekvensseihin.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Rakennetaan merkkisanasto

Rakentaaksemme merkkitason generatiivisen verkon, meidän täytyy jakaa teksti yksittäisiin merkkeihin sanojen sijaan. `TextVectorization`-kerros, jota olemme aiemmin käyttäneet, ei pysty tähän, joten meillä on kaksi vaihtoehtoa:

* Ladata teksti manuaalisesti ja tehdä tokenisointi "käsin", kuten [tässä Kerasin virallisessa esimerkissä](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Käyttää `Tokenizer`-luokkaa merkkitason tokenisointiin.

Valitsemme toisen vaihtoehdon. `Tokenizer`-luokkaa voidaan myös käyttää sanojen tokenisointiin, joten sen avulla voi helposti vaihtaa merkkitason ja sanatasoisen tokenisoinnin välillä.

Merkkitason tokenisointia varten meidän täytyy välittää parametri `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Haluamme myös käyttää yhtä erityistä tokenia merkitsemään **sekvenssin loppua**, jota kutsumme nimellä `<eos>`. Lisätään se manuaalisesti sanastoon:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

Nyt, tekstin koodaamiseksi numerosarjoiksi, voimme käyttää:


In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Generatiivisen RNN:n kouluttaminen otsikoiden luomiseen

Tapa, jolla koulutamme RNN:n luomaan uutisotsikoita, on seuraava. Jokaisella askeleella otamme yhden otsikon, joka syötetään RNN:ään, ja jokaiselle syötehahmolle pyydämme verkkoa tuottamaan seuraavan ulostulohahmon:

![Kuva, joka näyttää esimerkin RNN:n sanan 'HELLO' generoinnista.](../../../../../translated_images/fi/rnn-generate.56c54afb52f9781d.webp)

Sekvenssimme viimeisen hahmon kohdalla pyydämme verkkoa tuottamaan `<eos>`-tokenin.

Suurin ero generatiivisen RNN:n välillä, jota käytämme tässä, on se, että otamme ulostulon jokaiselta RNN:n askeleelta, emmekä vain viimeisestä solusta. Tämä voidaan saavuttaa määrittämällä `return_sequences`-parametri RNN-solulle.

Näin ollen koulutuksen aikana verkon syöte olisi tietyn pituisen koodattujen hahmojen sekvenssi, ja ulostulo olisi saman pituisen sekvenssi, mutta siirretty yhdellä elementillä ja päätetty `<eos>`-tokenilla. Minibatch koostuu useista tällaisista sekvensseistä, ja meidän täytyy käyttää **täydennystä** kaikkien sekvenssien tasaamiseksi.

Luodaan funktiot, jotka muuntavat datan meille. Koska haluamme täyttää sekvenssit minibatch-tasolla, ryhmittelemme ensin datan kutsumalla `.batch()`, ja sitten `map`-toiminnolla teemme muunnoksen. Muunnosfunktio ottaa siis kokonaisen minibatchin parametrina:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Muutamia tärkeitä asioita, joita teemme tässä:
* Ensin eristämme varsinaisen tekstin merkkijonotensorista
* `text_to_sequences` muuntaa merkkijonojen listan kokonaislukutensoreiden listaksi
* `pad_sequences` täyttää nämä tensorit niiden maksimi pituuteen
* Lopuksi teemme yhden kuuman koodauksen kaikille merkeille, sekä siirron ja `<eos>`-lisäyksen. Pian näemme, miksi tarvitsemme yhden kuuman koodattuja merkkejä

Tämä funktio on kuitenkin **Python-tyylinen**, eli sitä ei voida automaattisesti muuntaa Tensorflow'n laskentagrafiksi. Saamme virheitä, jos yritämme käyttää tätä funktiota suoraan `Dataset.map`-funktiossa. Meidän täytyy sulkea tämä Python-tyylinen kutsu käyttämällä `py_function`-käärettä:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Huomio**: Python- ja Tensorflow-muunnostoimintojen erottaminen toisistaan saattaa tuntua hieman monimutkaiselta, ja saatat pohtia, miksi emme muunna datasettiä tavallisilla Python-funktioilla ennen sen syöttämistä `fit`-funktiolle. Vaikka tämä onkin mahdollista, `Dataset.map`-funktion käyttö tarjoaa suuren edun, koska datan muunnosputki suoritetaan Tensorflow'n laskentakaavion avulla. Tämä hyödyntää GPU-laskentaa ja minimoi datan siirtämisen tarpeen CPU:n ja GPU:n välillä.

Nyt voimme rakentaa generaattoriverkkomme ja aloittaa koulutuksen. Se voi perustua mihin tahansa rekursiiviseen soluun, joita käsittelimme edellisessä osiossa (yksinkertainen, LSTM tai GRU). Esimerkissämme käytämme LSTM:ää.

Koska verkko ottaa syötteenä merkkejä ja sanaston koko on melko pieni, emme tarvitse upotustasoa (embedding layer). Yksinkertaisesti one-hot-koodattu syöte voidaan syöttää suoraan LSTM-soluun. Ulostulokerros olisi `Dense`-luokittelija, joka muuntaa LSTM:n ulostulon one-hot-koodatuiksi token-numeroiksi.

Lisäksi, koska käsittelemme vaihtelevan pituisia sekvenssejä, voimme käyttää `Masking`-kerrosta luomaan maskin, joka ohittaa merkkijonon täytetyt osat. Tämä ei ole ehdottoman välttämätöntä, koska emme ole erityisen kiinnostuneita kaikesta, mikä tulee `<eos>`-tokenin jälkeen, mutta käytämme sitä saadaksemme kokemusta tämän tyyppisestä kerroksesta. `input_shape` olisi `(None, vocab_size)`, missä `None` osoittaa vaihtelevan pituisen sekvenssin, ja ulostulon muoto on myös `(None, vocab_size)`, kuten voit nähdä `summary`-tulosteesta:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Tuloksen luominen

Nyt kun olemme kouluttaneet mallin, haluamme käyttää sitä tuottamaan jonkinlaista tulosta. Ensinnäkin tarvitsemme tavan purkaa tekstin, joka on esitetty token-numeroiden sekvenssinä. Tähän voisimme käyttää `tokenizer.sequences_to_texts` -funktiota; se ei kuitenkaan toimi hyvin merkkitasoisessa tokenoinnissa. Siksi otamme sanaston tokenoijasta (nimeltään `word_index`), rakennamme käänteisen kartan ja kirjoitamme oman purkufunktion:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Aloitetaan generointi. Ensin otamme jonkin merkkijonon `start`, koodaamme sen sekvenssiksi `inp`, ja sitten jokaisessa vaiheessa kutsumme verkkoamme ennustamaan seuraavan merkin.

Verkon tulos `out` on vektori, jossa on `vocab_size` elementtiä, jotka edustavat kunkin tokenin todennäköisyyksiä. Voimme löytää todennäköisimmän tokenin numeron käyttämällä `argmax`. Tämän jälkeen lisäämme tämän merkin generoituun tokenien listaan ja jatkamme generointia. Tämä prosessi, jossa generoidaan yksi merkki, toistetaan `size` kertaa, jotta saadaan tarvittava määrä merkkejä, ja lopetamme aikaisemmin, jos `eos_token` kohdataan.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Näytteen ottaminen koulutuksen aikana

Koska meillä ei ole käytettävissä hyödyllisiä mittareita, kuten *tarkkuutta*, ainoa tapa nähdä, että mallimme paranee, on **ottaa näytteitä** luoduista merkkijonoista koulutuksen aikana. Tätä varten käytämme **takaisinkutsuja** eli funktioita, jotka voimme välittää `fit`-funktiolle ja jotka kutsutaan säännöllisin väliajoin koulutuksen aikana.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Tämä esimerkki tuottaa jo melko hyvää tekstiä, mutta sitä voidaan parantaa useilla tavoilla:

* **Enemmän tekstiä**. Olemme käyttäneet tehtävässämme vain otsikoita, mutta voit kokeilla koko tekstin käyttöä. Muista, että RNN:t eivät ole kovin hyviä käsittelemään pitkiä sekvenssejä, joten on järkevää joko jakaa ne lyhyempiin lauseisiin tai aina kouluttaa kiinteällä sekvenssipituudella, joka on ennalta määritelty arvo `num_chars` (esimerkiksi 256). Voit yrittää muuttaa yllä olevan esimerkin tällaiseksi arkkitehtuuriksi käyttämällä [virallista Keras-opasta](https://keras.io/examples/generative/lstm_character_level_text_generation/) inspiraationa.

* **Monikerroksinen LSTM**. Kannattaa kokeilla 2 tai 3 kerrosta LSTM-soluja. Kuten mainitsimme edellisessä osiossa, jokainen LSTM-kerros tunnistaa tiettyjä tekstin piirteitä, ja merkkitason generaattorin tapauksessa voimme odottaa alemman LSTM-tason vastaavan tavujen tunnistamisesta ja ylempien tasojen sanojen ja sanayhdistelmien tunnistamisesta. Tämä voidaan toteuttaa yksinkertaisesti antamalla kerrosten lukumäärä -parametri LSTM:n rakentajalle.

* Voit myös kokeilla **GRU-yksiköitä** ja katsoa, mitkä toimivat paremmin, sekä **erilaisia piilotettujen kerrosten kokoja**. Liian suuri piilotettu kerros voi johtaa ylisovittamiseen (esim. verkko oppii tarkan tekstin), kun taas liian pieni koko ei ehkä tuota hyviä tuloksia.


## Pehmeä tekstin generointi ja lämpötila

Aiemmassa `generate`-määritelmässä valitsimme aina seuraavaksi merkiksi sen, jolla oli korkein todennäköisyys. Tämä johti usein siihen, että teksti "kierrätti" samoja merkkijonoja yhä uudelleen, kuten tässä esimerkissä:
```
today of the second the company and a second the company ...
```

Jos kuitenkin tarkastelemme seuraavan merkin todennäköisyysjakaumaa, voi olla, että muutaman korkeimman todennäköisyyden ero ei ole suuri, esimerkiksi yksi merkki voi olla todennäköisyydellä 0.2 ja toinen 0.19, jne. Esimerkiksi, kun etsitään seuraavaa merkkiä jaksossa '*play*', seuraava merkki voi yhtä hyvin olla joko välilyönti tai **e** (kuten sanassa *player*).

Tämä johtaa siihen päätelmään, että ei ole aina "reilua" valita merkkiä, jolla on korkeampi todennäköisyys, koska toisen korkein todennäköisyys voi silti tuottaa merkityksellistä tekstiä. Onkin viisaampaa **näytteistää** merkkejä verkon tuottaman todennäköisyysjakauman perusteella.

Tämä näytteistäminen voidaan toteuttaa `np.multinomial`-funktiolla, joka toteuttaa niin kutsutun **multinomiaalisen jakauman**. Alla on määritelty funktio, joka toteuttaa tämän **pehmeän** tekstin generoinnin:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Olemme ottaneet käyttöön yhden lisäparametrin nimeltä **lämpötila**, jota käytetään osoittamaan, kuinka tiukasti meidän tulisi noudattaa korkeinta todennäköisyyttä. Jos lämpötila on 1.0, teemme tasapuolista multinomiaalista otantaa, ja kun lämpötila kasvaa äärettömyyteen - kaikki todennäköisyydet muuttuvat yhtä suuriksi, ja valitsemme seuraavan merkin satunnaisesti. Alla olevassa esimerkissä voimme havaita, että teksti muuttuu merkityksettömäksi, kun nostamme lämpötilaa liikaa, ja se muistuttaa "kierrätettyä" tiukasti generoituvaa tekstiä, kun lämpötila lähestyy arvoa 0.



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäinen asiakirja sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
